In [9]:
import random
from typing import Literal

import gnews
import newspaper
import requests
from pydantic import BaseModel


class Article(BaseModel):
    topic: str

    n: int = 10
    language: Literal['en', 'de', 'nl', 'sr', 'it'] = 'en'
    country: Literal['US', 'DE', 'NL', 'RS', 'IT'] = 'US'

    url: str = None
    title: str = None
    summary: str = None

    def __init__(self, **data):
        super().__init__(**data)

        news_page = self.retrieve_news_page()
        news = random.choice(news_page)

        self.url = requests.get(news['url']).url

        article = self.retrieve_article()

        self.title = article.title
        self.summary = article.summary.replace('\n', ' ')

    def __call__(self) -> dict:
        return {
            'url': article.url,
            'title': article.title,
            'summary': article.summary.replace('\n', ' ')
        }

    def retrieve_news_page(self):
        return gnews.GNews(
            max_results=self.n,
            language=self.language,
            country=self.country
        ).get_news(self.topic)

    def retrieve_article(self):
        article = newspaper.Article(self.url)
        article.download()
        article.parse()
        article.nlp()

        return article

In [11]:
article = Article(topic='Germany')
article()

{'url': 'https://www.dw.com/en/germany-rethinks-bundeswehr-and-deterrence/a-68248353',
 'title': 'Germany hopes to boost military and deterrence – DW – 02',
 'summary': 'Nevertheless, in 2014, NATO defense ministers agreed to commit a minimum of 2% of their country\'s gross domestic product (GDP) to defense. There are a huge range of views on nuclear weapons among the 27 EU member states. Historically, Germany\'s self-imposed prohibition on nuclear weapons is tied up with the post-World-War-II order and West Germany\'s role in NATO defense strategy. In order to limit proliferation, the US government granted West Germany security guarantees and some role in providing the planes for delivering of nuclear weapons. "In the current context, I see no reason why Germany should acquire nuclear weapons itself, and there are good reasons not to," said ECFR analyst Loss.'}